<a href="https://colab.research.google.com/github/jennyjtang/110-projects/blob/main/zor_viz_prototypes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#imports
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
df = pd.read_csv("zor_prelim_data.csv")
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna(subset=['Date'])
df.sort_values('Date', inplace=True)
df.fillna(method='ffill', inplace=True)

<ipython-input-4-2a73be02079c>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
<ipython-input-4-2a73be02079c>:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [ ]:
#read generated seizure data file
df = pd.read_csv("seizure_data.csv")
print(df.head())

   Record_ID patient_id        Date  Seizure_Frequency  \
0          1      P0303  2025-03-28                  1   
1          2      P0971  2025-03-13                  0   
2          3      P0825  2025-04-02                  0   
3          4      P0118  2025-01-23                  1   
4          5      P0813  2025-03-09                  1   

   Medication_Adherence (%)  Heart_Rate (BPM)  Oxygen_Level (%)  \
0                        70             113.0              92.0   
1                        59               NaN               NaN   
2                        90               NaN               NaN   
3                        89              87.0              97.0   
4                        68             141.0              97.0   

   Seizure_Duration (sec)  Post-Seizure_Recovery_Time (min) Seizure_Location  \
0                    25.0                              14.0             Work   
1                     NaN                               NaN             Home   
2       

In [ ]:
#standardize values
df["patient_id"] = df["patient_id"].str.replace("P", "", regex=True).astype(int)

In [ ]:
#looking at a specific patient
patient_id = 303
patient_data = df[df["patient_id"] == patient_id]

#check if pateient data exists
if patient_data.empty:
    print(f"No data found for Patient {patient_id}. Try another ID.")
else:
    print(patient_data.head())

      Record_ID  patient_id        Date  Seizure_Frequency  \
0             1         303  2025-03-28                  1   
1110       1111         303  2025-01-17                  1   
1349       1350         303  2025-02-05                  3   
1786       1787         303  2025-03-05                  3   
2388       2389         303  2025-01-03                  1   

      Medication_Adherence (%)  Heart_Rate (BPM)  Oxygen_Level (%)  \
0                           70             113.0              92.0   
1110                        98             133.0              98.0   
1349                        93              90.0              93.0   
1786                        65             141.0              87.0   
2388                        63             120.0              94.0   

      Seizure_Duration (sec)  Post-Seizure_Recovery_Time (min)  \
0                       25.0                              14.0   
1110                    21.0                              16.0   
1349    

In [ ]:
# make and check that date values are datetime
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Record_ID                         10000 non-null  int64         
 1   patient_id                        10000 non-null  int64         
 2   Date                              10000 non-null  datetime64[ns]
 3   Seizure_Frequency                 10000 non-null  int64         
 4   Medication_Adherence (%)          10000 non-null  int64         
 5   Heart_Rate (BPM)                  5910 non-null   float64       
 6   Oxygen_Level (%)                  5910 non-null   float64       
 7   Seizure_Duration (sec)            5910 non-null   float64       
 8   Post-Seizure_Recovery_Time (min)  5910 non-null   float64       
 9   Seizure_Location                  10000 non-null  object        
 10  Emergency_Intervention            5910 non-null

In [ ]:
#group all data by year-month
df["month"] = df["Date"].dt.to_period("M")

In [ ]:
# Convert Period objects to datetime objects (first day of each month)
# This assumes seizure_trend.index contains period objects
x_values = [pd.Period(period).to_timestamp() for period in seizure_trend.index]

# Create the plotly figure
fig = go.Figure()

# Add the line plot with markers
fig.add_trace(
    go.Scatter(
        x=x_values,
        y=seizure_trend.values,
        mode='lines+markers',
        marker=dict(color='red', size=8),
        line=dict(color='red', width=2)
    )
)

# Update layout with title and axis labels
fig.update_layout(
    title="Seizure Trends Over Time",
    xaxis_title="Month",
    yaxis_title="Number of Seizures",
    template="plotly_white",
    width=1000,
    height=500
)

# Format x-axis to display month and year
fig.update_xaxes(
    tickformat="%b %Y",  # Format as "Jan 2023", "Feb 2023", etc.
    dtick="M1",          # Set tick marks to appear monthly
    ticklabelmode="period"  # Use period mode for cleaner display
)

# Show the interactive plot
fig.show()

Graph 1. General Trendline of All Patients

In [ ]:
patient_id = 303
patient_data = df[df["patient_id"] == patient_id]

# Group by Date to get daily seizure counts
daily_seizures = patient_data.groupby("Date")["Seizure_Frequency"].sum().reset_index()

# Create complete date range (fill missing dates with zeros)
date_range = pd.date_range(start=daily_seizures["Date"].min(), end=daily_seizures["Date"].max())
complete_daily = pd.DataFrame({"Date": date_range})
complete_daily = complete_daily.merge(daily_seizures, on="Date", how="left").fillna(0)

# Create the plotly figure
fig = go.Figure()

# Add the line plot with markers
fig.add_trace(
    go.Scatter(
        x=complete_daily["Date"],
        y=complete_daily["Seizure_Frequency"],
        mode='lines+markers',
        marker=dict(color='blue', size=6),
        line=dict(color='blue', width=2),
        name=f'Patient {patient_id}'
    )
)

# Add markers for actual seizure events
fig.add_trace(
    go.Scatter(
        x=daily_seizures[daily_seizures["Seizure_Frequency"] > 0]["Date"],
        y=daily_seizures[daily_seizures["Seizure_Frequency"] > 0]["Seizure_Frequency"],
        mode='markers',
        marker=dict(color='red', size=8, symbol='circle'),
        name='Seizure Events',
        hovertemplate='Date: %{x}<br>Seizures: %{y}<extra></extra>'
    )
)

# Update layout with title and axis labels
fig.update_layout(
    title=f"Daily Seizure Activity for Patient {patient_id}",
    xaxis_title="Date",
    yaxis_title="Number of Seizures",
    template="plotly_white",
    width=1000,
    height=500,
    hovermode="x unified",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

# Format x-axis to show dates but organize by month
fig.update_xaxes(
    tickformat="%b %d, %Y",
    dtick="M1",  # Monthly tick marks
    ticklabelmode="period"
)

# Show the interactive plot
fig.show()

Graph 2.1 Interactive Seizure Count for Patient 303 (Line)

In [ ]:
patient_id = 303
patient_data = df[df["patient_id"] == patient_id]

# Group by Date to get daily seizure counts
daily_seizures = patient_data.groupby("Date")["Seizure_Frequency"].sum().reset_index()

# Create complete date range (fill missing dates with zeros)
date_range = pd.date_range(start=daily_seizures["Date"].min(), end=daily_seizures["Date"].max())
complete_daily = pd.DataFrame({"Date": date_range})
complete_daily = complete_daily.merge(daily_seizures, on="Date", how="left").fillna(0)

# Create the plotly bar chart
fig = go.Figure()

# Add bars for seizure frequency
fig.add_trace(
    go.Bar(
        x=complete_daily["Date"],
        y=complete_daily["Seizure_Frequency"],
        marker_color='blue',
        name=f'Patient {patient_id}',
        hovertemplate='Date: %{x}<br>Seizures: %{y}<extra></extra>'
    )
)

# Update layout with title and axis labels
fig.update_layout(
    title=f"Daily Seizure Activity for Patient {patient_id}",
    xaxis_title="Date",
    yaxis_title="Number of Seizures",
    template="plotly_white",
    width=1000,
    height=500,
    hovermode="x unified"
)

# Configure x-axis with adaptive tick spacing
fig.update_xaxes(
    tickformatstops=[
        dict(dtickrange=[None, 1000*60*60*24*30], value="%b %d\n%Y"),  # Daily view when zoomed in (30 days or less)
        dict(dtickrange=[1000*60*60*24*30, 1000*60*60*24*90], value="%b %d"),  # Days with month when zoomed to ~1-3 months
        dict(dtickrange=[1000*60*60*24*90, None], value="%b %Y")  # Just months when zoomed out
    ],
    ticklabelmode="period",
    autorange=True
)

# Add range slider for easier navigation through the timeline
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=7, label="1w", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date"
    )
)

# Show the interactive plot
fig.show()

Graph 2.2 Interactive Seizure Count for Patient 303 (Bar)

In [ ]:
patient_id = 321
patient_data = df[df["patient_id"] == patient_id]

# Group by Date to get daily seizure counts
daily_seizures = patient_data.groupby("Date")["Seizure_Frequency"].sum().reset_index()

# Create complete date range (fill missing dates with zeros)
date_range = pd.date_range(start=daily_seizures["Date"].min(), end=daily_seizures["Date"].max())
complete_daily = pd.DataFrame({"Date": date_range})
complete_daily = complete_daily.merge(daily_seizures, on="Date", how="left").fillna(0)

# Create the plotly bar chart
fig = go.Figure()

# Add bars for seizure frequency
fig.add_trace(
    go.Bar(
        x=complete_daily["Date"],
        y=complete_daily["Seizure_Frequency"],
        marker_color='blue',
        name=f'Patient {patient_id}',
        hovertemplate='Date: %{x}<br>Seizures: %{y}<extra></extra>'
    )
)

# Update layout with title and axis labels
fig.update_layout(
    title=f"Daily Seizure Activity for Patient {patient_id}",
    xaxis_title="Date",
    yaxis_title="Number of Seizures",
    template="plotly_white",
    width=1000,
    height=500,
    hovermode="x unified"
)

# Configure x-axis with adaptive tick spacing
fig.update_xaxes(
    tickformatstops=[
        dict(dtickrange=[None, 1000*60*60*24*30], value="%b %d\n%Y"),  # Daily view when zoomed in (30 days or less)
        dict(dtickrange=[1000*60*60*24*30, 1000*60*60*24*90], value="%b %d"),  # Days with month when zoomed to ~1-3 months
        dict(dtickrange=[1000*60*60*24*90, None], value="%b %Y")  # Just months when zoomed out
    ],
    ticklabelmode="period",
    autorange=True
)

# Add range slider for easier navigation through the timeline
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=7, label="1w", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date"
    )
)

# Show the interactive plot
fig.show()

Graph 3 Interactive Seizure Count for Patient 321 (Bar)

In [ ]:
patient_id = 988
patient_data = df[df["patient_id"] == patient_id]

# Group by Date to get daily seizure counts
daily_seizures = patient_data.groupby("Date")["Seizure_Frequency"].sum().reset_index()

# Create complete date range (fill missing dates with zeros)
date_range = pd.date_range(start=daily_seizures["Date"].min(), end=daily_seizures["Date"].max())
complete_daily = pd.DataFrame({"Date": date_range})
complete_daily = complete_daily.merge(daily_seizures, on="Date", how="left").fillna(0)

# Create the plotly bar chart
fig = go.Figure()

# Add bars for seizure frequency
fig.add_trace(
    go.Bar(
        x=complete_daily["Date"],
        y=complete_daily["Seizure_Frequency"],
        marker_color='blue',
        name=f'Patient {patient_id}',
        hovertemplate='Date: %{x}<br>Seizures: %{y}<extra></extra>'
    )
)

# Update layout with title and axis labels
fig.update_layout(
    title=f"Daily Seizure Activity for Patient {patient_id}",
    xaxis_title="Date",
    yaxis_title="Number of Seizures",
    template="plotly_white",
    width=1000,
    height=500,
    hovermode="x unified"
)

# Configure x-axis with adaptive tick spacing
fig.update_xaxes(
    tickformatstops=[
        dict(dtickrange=[None, 1000*60*60*24*30], value="%b %d\n%Y"),  # Daily view when zoomed in (30 days or less)
        dict(dtickrange=[1000*60*60*24*30, 1000*60*60*24*90], value="%b %d"),  # Days with month when zoomed to ~1-3 months
        dict(dtickrange=[1000*60*60*24*90, None], value="%b %Y")  # Just months when zoomed out
    ],
    ticklabelmode="period",
    autorange=True
)

# Add range slider for easier navigation through the timeline
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=7, label="1w", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date"
    )
)

# Show the interactive plot
fig.show()

Graph 4 Interactive Seizure Count for Patient 988 (Bar)

In [ ]:
import plotly.graph_objects as go
from datetime import datetime

def create_seizure_bar_chart(df):
    # Create the figure
    fig = go.Figure()

    # Add the bar trace
    fig.add_trace(
        go.Bar(
            x=df['Date'],
            y=df['Seizure Occurences'],
            name='Seizure Occurrences',
            marker_color='red',
            opacity=0.7,
            hovertemplate='Date: %{x|%b %d, %Y}<br>Seizures: %{y}<extra></extra>'
        )
    )

    # Update layout with dynamic date formatting
    fig.update_layout(
        title='Monthly Seizure Occurrences: Zor',
        xaxis_title="Date",
        yaxis_title="Number of Seizures",
        template="plotly_white",
        width=1000,
        height=500,
        hovermode="x unified",
        xaxis=dict(
            tickformatstops = [
                dict(dtickrange=[None, 86400000], value="%b %d, %Y %H:%M"),  # <1 day: show full date+time
                dict(dtickrange=[86400000, 604800000], value="%b %d, %Y"),    # 1 day - 1 week: show date
                dict(dtickrange=[604800000, None], value="%b %Y")              # > 1 week: show month/year
            ],
            type='date',
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1d", step="day", stepmode="backward"),
                    dict(count=7, label="1w", step="day", stepmode="backward"),
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=3, label="3m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all", label="All")
                ])
            ),
            rangeslider=dict(visible=True)
        )
    )

    return fig

# Usage:
fig = create_seizure_bar_chart(df)
fig.show()

Graph 5 Interactive Seizure Count for Zor (Bar)

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load and prepare the data
df = pd.read_csv('zor_prelim_data.csv')
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna(subset=['Date'])
df.sort_values('Date', inplace=True)
df.fillna(method='ffill', inplace=True)

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add seizure occurrences as bars (primary y-axis)
fig.add_trace(
    go.Bar(
        x=df['Date'],
        y=df['Seizure Occurences'],
        name='Seizure Occurrences',
        marker_color='red',
        opacity=0.6,
        hovertemplate='Date: %{x}<br>Seizures: %{y}<extra></extra>'
    ),
    secondary_y=False
)

# Add medication lines (secondary y-axis)
medications = ['Topomax (mg)', 'Keppra (mg)', 'Xcopiri (mg)', 'Fentapla (ml)', 'Epidiolux (ml)']
colors = ['blue', 'green', 'purple', 'orange', 'brown']
visibility = [True, True, True, True, True]  # All medications visible by default

for med, color in zip(medications, colors):
    fig.add_trace(
        go.Scatter(
            x=df['Date'],
            y=df[med],
            name=med,
            line=dict(color=color, width=2),
            hovertemplate='Date: %{x}<br>Dosage: %{y}<extra></extra>',
            connectgaps=True,
            visible=True  # Initially visible
        ),
        secondary_y=True
    )

# Create buttons for toggling medications
med_buttons = []
for i, med in enumerate(medications):
    visibility = [False] * len(medications)
    visibility[i] = True  # Only show the selected medication

    med_buttons.append(
        dict(
            label=med,
            method="update",
            args=[{"visible": [True] + visibility},  # First True is for seizure bar
                 {"title": f"Seizure Occurrences and {med} Dosage Over Time"}]
        )
    )

# Create "Show All" button
show_all_button = dict(
    label="Show All",
    method="update",
    args=[{"visible": [True] + [True]*len(medications)},
          {"title": "Seizure Occurrences and All Medication Dosages Over Time"}]
)

# Create "Hide All" button
hide_all_button = dict(
    label="Hide All",
    method="update",
    args=[{"visible": [True] + [False]*len(medications)},
          {"title": "Seizure Occurrences Over Time"}]
)

# Add dropdown menu for medication selection
fig.update_layout(
    updatemenus=[
        dict(
            type="dropdown",
            direction="down",
            x=1.1,
            y=1.15,
            showactive=True,
            buttons=[show_all_button, hide_all_button] + med_buttons,
        )
    ]
)

# Add figure title and axis labels
fig.update_layout(
    title_text='Seizure Occurrences and Medication Dosages Over Time',
    width=1200,
    height=600,
    hovermode='x unified',
    template='plotly_white',
    margin=dict(r=150)  # Add right margin for dropdown
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Seizure</b> Occurrences", secondary_y=False)
fig.update_yaxes(title_text="<b>Medication</b> Dosage", secondary_y=True)

# Configure x-axis with adaptive tick spacing
fig.update_xaxes(
    tickformatstops=[
        dict(dtickrange=[None, 1000*60*60*24*30], value="%b %d\n%Y"),
        dict(dtickrange=[1000*60*60*24*30, 1000*60*60*24*90], value="%b %d"),
        dict(dtickrange=[1000*60*60*24*90, None], value="%b %Y")
    ],
    ticklabelmode="period"
)

# Add range slider for easier navigation
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=7, label="1w", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date"
    )
)

fig.show()

<ipython-input-13-4ecb82fce4f7>:7: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

<ipython-input-13-4ecb82fce4f7>:10: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



Graph 6.1 Medication Display Test 1

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load and prepare the data
df = pd.read_csv('zor_prelim_data.csv')
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna(subset=['Date'])
df.sort_values('Date', inplace=True)
df.fillna(method='ffill', inplace=True)

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add seizure occurrences as dots (will be positioned above bars)
seizure_days = df[df['Seizure Occurences'] > 0]
fig.add_trace(
    go.Scatter(
        x=seizure_days['Date'],
        y=[df['Topomax (mg)'].max()*1.1]*len(seizure_days),  # Position above bars
        name='Seizure Occurrence',
        mode='markers',
        marker=dict(
            symbol='diamond',
            size=10,
            color='red',
            line=dict(width=1, color='black')
        ),
        hovertemplate='Seizure on %{x|%b %d, %Y}<extra></extra>',
    ),
    secondary_y=False
)

# Add medication bars (stacked)
medications = ['Topomax (mg)', 'Keppra (mg)', 'Xcopiri (mg)', 'Fentapla (ml)', 'Epidiolux (ml)']
colors = ['#1f77b4', '#2ca02c', '#9467bd', '#ff7f0e', '#8c564b']  # Distinct colors

for med, color in zip(medications, colors):
    fig.add_trace(
        go.Bar(
            x=df['Date'],
            y=df[med],
            name=med,
            marker_color=color,
            opacity=0.7,
            hovertemplate='Date: %{x|%b %d, %Y}<br>%{text}: %{y}<extra></extra>',
            text=[med.split(' ')[0]]*len(df),
            base=0 if med == medications[0] else None
        ),
        secondary_y=False
    )

# Create toggle switch between views
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="left",
            x=0.1,
            xanchor="left",
            y=1.15,
            yanchor="top",
            buttons=[
                dict(
                    label="Stacked Medications",
                    method="update",
                    args=[{"visible": [True] + [True]*len(medications)},
                          {"title": "Medication Dosages (Stacked) with Seizure Indicators",
                           "barmode": "stack"}]
                ),
                dict(
                    label="Original View",
                    method="update",
                    args=[{"visible": [True] + [False]*len(medications)},
                          {"title": "Seizure Occurrences Only",
                           "barmode": "group"}]
                )
            ]
        )
    ]
)

# Add figure title and axis labels
fig.update_layout(
    title_text='Medication Dosages (Stacked) with Seizure Indicators',
    width=1200,
    height=600,
    hovermode='x unified',
    template='plotly_white',
    barmode='stack',
    margin=dict(r=150, t=100),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Medication</b> Dosage", secondary_y=False)
fig.update_yaxes(showgrid=False, secondary_y=True)

# Configure x-axis with adaptive tick spacing
fig.update_xaxes(
    tickformatstops=[
        dict(dtickrange=[None, 1000*60*60*24*30], value="%b %d\n%Y"),
        dict(dtickrange=[1000*60*60*24*30, 1000*60*60*24*90], value="%b %d"),
        dict(dtickrange=[1000*60*60*24*90, None], value="%b %Y")
    ],
    ticklabelmode="period"
)

# Add range slider for easier navigation
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=7, label="1w", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date"
    )
)

fig.show()

<ipython-input-17-4208c609e306>:7: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

<ipython-input-17-4208c609e306>:10: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



Graph 6.2 Medication Display Test 2

In [ ]:
# Calculate max values for axis scaling
max_meds = df[['Topomax (mg)', 'Keppra (mg)', 'Xcopiri (mg)', 'Fentapla (ml)', 'Epidiolux (ml)']].sum(axis=1).max()
max_seizures = df['Seizure Occurences'].max()
axis_ratio = max_meds / max_seizures if max_seizures > 0 else 1

# Create figure with proportional secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add stacked medication bars (primary axis)
meds = ['Topomax (mg)', 'Keppra (mg)', 'Xcopiri (mg)', 'Fentapla (ml)', 'Epidiolux (ml)']
colors = ['#4C78A8', '#54A24B', '#E45756', '#F58518', '#72B7B2']

for med, color in zip(meds, colors):
    fig.add_trace(
        go.Bar(
            x=df['Date'],
            y=df[med],
            name=med.split(' (')[0],
            marker_color=color,
            opacity=0.85,
            hovertemplate='%{x|%b %d, %Y}<br>%{text}: %{y}<extra></extra>',
            text=[med.split(' ')[0]]*len(df)
        ),
        secondary_y=False
    )

# Add seizure line (secondary axis)
fig.add_trace(
    go.Scatter(
        x=df['Date'],
        y=df['Seizure Occurences'] * axis_ratio,  # Scale to match medication axis
        name='Seizures',
        line=dict(color='#1F77B4', width=3),
        mode='lines+markers',
        marker=dict(size=8, symbol='diamond'),
        hovertemplate='%{x|%b %d, %Y}<br>Seizures: %{customdata}',
        customdata=df['Seizure Occurences']  # Show original seizure count in hover
    ),
    secondary_y=True
)

# Update layout with proportional scaling
fig.update_layout(
    title='<b>Medication Dosages vs Seizure Frequency</b><br><sub>Stacked bars show daily medication totals</sub>',
    barmode='stack',
    width=1200,
    height=650,
    hovermode='x unified',
    template='plotly_white',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    margin=dict(t=120, r=50),
    plot_bgcolor='rgba(248,248,248,0.9)'
)

# Configure axes
fig.update_yaxes(
    title_text="<b>Total Medication Dosage</b> (mg/ml)",
    secondary_y=False,
    rangemode='tozero'
)

fig.update_yaxes(
    title_text="<b>Seizure Count</b>",
    secondary_y=True,
    tickvals=[i * axis_ratio for i in range(int(max_seizures)+2)],
    ticktext=[str(i) for i in range(int(max_seizures)+2)],
    rangemode='tozero'
)

# Configure x-axis
fig.update_xaxes(
    title_text="Date",
    tickformat="%b %Y",
    rangeslider=dict(visible=True),
    rangeselector=dict(
        buttons=list([
            dict(count=7, label="1w", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=3, label="3m", step="month", stepmode="backward"),
            dict(step="all", label="Full range")
        ])
    )
)

fig.show()

In [ ]:
Graph 6.3 Medication Display Test 3